In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers.modeling_outputs import MaskedLMOutput
from datasets import load_dataset

In [ ]:
dataset = load_dataset("KevinZ/oLMpics", name="Age_Comparison")
dataset

In [ ]:
dataset["train"].features["answerKey"]

In [ ]:
dataset["train"][-1]

In [ ]:
choices = dataset["train"][0]["choices"]
choices_ids = tokenizer(choices, return_tensors="pt").input_ids
choices_ids = choices_ids[:, 1]
# remove 

choices_dict = {
    "values": choices,
    "ids": choices_ids
}

choices_dict

In [ ]:
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

In [ ]:
# Get the mask token and its id
tokenizer.mask_token, tokenizer.mask_token_id

In [ ]:
# Get the model's inputs as a PyTorch tensor
inputs = tokenizer(dataset["train"][0]["stem"], return_tensors="pt")
inputs

In [ ]:
# Get the position of the [MASK] token in the sentence
mask_pos = (inputs.input_ids.squeeze() == tokenizer.mask_token_id).nonzero(as_tuple=True)
mask_pos = int(mask_pos[0].squeeze())
mask_pos

In [ ]:
# predicts logits for each output token in the sentence. 
with torch.no_grad():
    output: MaskedLMOutput = model(**inputs)

In [ ]:
output.logits.shape # shape: (batch_size, seq_length, vocab_size)

In [ ]:
# get the logits for the token ids of the availaböe choices
choices_logits = torch.tensor(list(map(lambda id: output.logits[0, mask_pos, id], choices_dict["ids"])))
choices_logits

In [ ]:
# get the index of the choice with the maximum logit value
idx_max = int(choices_logits.argmax())
idx_max

In [ ]:
# check whether model prediction matches the correct answer
dataset["train"][0]["answerKey"] == idx_max